In [2]:
import tensorflow as tf
import numpy as np
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.14.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [4]:
import os
import shutil

# Define the paths to the Droughts and Floods folders
droughts_folder = "/home/chescore/Pictures/Train/Droughts"
floods_folder = "/home/chescore/Pictures/Train/Floods"

# Define the paths to the output directories for dry season and rainy season
dry_season_output = "/home/chescore/Documents/Hackathon/Train/Dry"
rainy_season_output = "/home/chescore/Documents/Hackathon/Train/Rainy"

# Create output directories if they don't exist
os.makedirs(dry_season_output, exist_ok=True)
os.makedirs(rainy_season_output, exist_ok=True)

# Function to classify images and move them to appropriate folders
def classify_images(folder_path, output_path, label):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
            source_file = os.path.join(folder_path, filename)
            destination_file = os.path.join(output_path, filename)
            shutil.copyfile(source_file, destination_file)
            # If you want to rename files with a label, you can do it here
            # shutil.move(source_file, os.path.join(output_path, f"{label}_{filename}"))

# Classify images in Droughts folder as 'dry season'
classify_images(droughts_folder, dry_season_output, 'dry_season')

# Classify images in Floods folder as 'rainy season'
classify_images(floods_folder, rainy_season_output, 'rainy_season')

print("Dataset creation complete.")


Dataset creation complete.


In [5]:
droughts_folder = "/home/chescore/Pictures/Validation/Droughts"
floods_folder = "/home/chescore/Pictures/Validation/Floods"

dry_season_output = "/home/chescore/Documents/Hackathon/Validation/Dry"
rainy_season_output = "/home/chescore/Documents/Hackathon/Validation/Rainy"

os.makedirs(dry_season_output, exist_ok=True)
os.makedirs(rainy_season_output, exist_ok=True)

def classify_images(folder_path, output_path, label):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
            source_file = os.path.join(folder_path, filename)
            destination_file = os.path.join(output_path, filename)
            shutil.copyfile(source_file, destination_file)

classify_images(droughts_folder, dry_season_output, 'dry_season')

classify_images(floods_folder, rainy_season_output, 'rainy_season')

print("Dataset creation complete.")

Dataset creation complete.


In [6]:
IMAGE_WIDTH, IMAGE_HEIGHT = 150, 150
BATCH_SIZE = 5

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/home/chescore/Documents/Hackathon/Train/',  
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary') 

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.samples // BATCH_SIZE, epochs=5)

Found 48 images belonging to 2 classes.


Epoch 1/5
9/9 [==============================] - 9s 630ms/step - loss: 0.7421 - accuracy: 0.7209
Epoch 2/5
9/9 [==============================] - 4s 486ms/step - loss: 0.3173 - accuracy: 0.8372
Epoch 3/5
9/9 [==============================] - 4s 476ms/step - loss: 0.1187 - accuracy: 0.9535
Epoch 4/5
9/9 [==============================] - 4s 476ms/step - loss: 0.1070 - accuracy: 0.9535
Epoch 5/5
9/9 [==============================] - 4s 481ms/step - loss: 0.0687 - accuracy: 0.9767


/home/chescore/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    '/home/chescore/Pictures/Validation',  # Path to your validation folder (contains 'dry_season' and 'rainy_season' subfolders)
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=1,
    class_mode='binary',  # Since we have two classes, it will be binary classification
    shuffle=False)  # Set shuffle to False for evaluation

# Evaluate the model
evaluation = model.evaluate(validation_generator)

print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])

Found 5 images belonging to 2 classes.


5/5 [==============================] - 1s 55ms/step - loss: 0.0193 - accuracy: 1.0000
Evaluation Loss: 0.0192601028829813
Evaluation Accuracy: 1.0


In [10]:
# Test the model on new data which is an image showing oncoming rainy seasons.
image_path = "/home/chescore/Documents/Hackathon/Test/Test 1.jpg"

# Preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

prediction = model.predict(img_array)

if prediction[0] < 0.5:
    print("Prediction: Dry Season")
    pred = 'Dry season'
else:
    print("Prediction: Rainy Season")
    pred = 'Rainy season'
    
print(pred)

1/1 [==============================] - 0s 137ms/step
Prediction: Rainy Season
Rainy season


In [11]:
from openai import OpenAI
client = OpenAI()

In [12]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a weather forecast assistant that can provide recommendations based on certain weather conditions such as rainy or dry seasons."},
    {"role": "user", "content": f"Given that it is a {pred}, write suggestions in which pastoralists can better prepare for such conditions."}
  ]
)

response = completion.choices[0].message.content

In [13]:
print(response)

During the rainy season, pastoralists can take several steps to better prepare for the wet weather and ensure the well-being of their livestock. Here are some recommendations:

1. Shelter: Ensure that there are adequate shelters for the animals to seek refuge from the rain. This could include constructing simple structures or using natural resources like trees for cover.

2. Vaccinations and healthcare: Prioritize healthcare for the animals by ensuring they are up-to-date on vaccinations and treatments to prevent diseases that may become more prevalent during the rainy season.

3. Feed and water management: Plan ahead for the availability of feed and clean drinking water for the animals during the rainy season, as the wet conditions may affect grazing areas and water sources.

4. Grazing management: Monitor the grazing areas regularly to prevent overgrazing and soil erosion, which can be exacerbated during the rainy season.

5. Emergency preparedness: Have a contingency plan in place f

In [14]:
first_three_points = response.strip().split("\n\n")[0:2]

# Printing the first three points
for point in first_three_points:
    res = point.strip()
    print(res)

During the rainy season, pastoralists can take several steps to better prepare for the wet weather and ensure the well-being of their livestock. Here are some recommendations:
1. Shelter: Ensure that there are adequate shelters for the animals to seek refuge from the rain. This could include constructing simple structures or using natural resources like trees for cover.


In [46]:
from twilio.rest import Client
account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='+18456849823',
  to='+254710499767',
  body=res
)

print(message.sid)

SM71fb65b4e282922f529c2f9b75528729
